In [ ]:
import cupy as cp
import time
N = 4096  
TILE = 16

# Allocate large random matrices
A = cp.random.rand(N, N, dtype=cp.float32)
B = cp.random.rand(N, N, dtype=cp.float32)

# Reference CuPy dot product (cuBLAS)
start = time.time()
C_normal = cp.dot(A, B)
cp.cuda.Stream.null.synchronize()
end = time.time()
time_normal = end - start
print(f"[Normal CuPy dot] Time: {time_normal:.4f} seconds")

# CUDA tiled kernel
kernel_code = f'''
extern "C" __global__
void matmul_tiled(const float* A, const float* B, float* C, int N) {{
    __shared__ float sA[{TILE}][{TILE}];
    __shared__ float sB[{TILE}][{TILE}];
    
    int row = blockIdx.y * {TILE} + threadIdx.y;
    int col = blockIdx.x * {TILE} + threadIdx.x;
    
    float value = 0.0f;
    
    for (int t = 0; t < (N + {TILE} - 1)/{TILE}; t++) {{
        int tiled_row = row;
        int tiled_col = t*{TILE} + threadIdx.x;
        if(tiled_row < N && tiled_col < N) sA[threadIdx.y][threadIdx.x] = A[tiled_row*N + tiled_col];
        else sA[threadIdx.y][threadIdx.x] = 0.0f;
        
        tiled_row = t*{TILE} + threadIdx.y;
        tiled_col = col;
        if(tiled_row < N && tiled_col < N) sB[threadIdx.y][threadIdx.x] = B[tiled_row*N + tiled_col];
        else sB[threadIdx.y][threadIdx.x] = 0.0f;
        
        __syncthreads();
        
        for(int k=0; k<{TILE}; k++) value += sA[threadIdx.y][k] * sB[k][threadIdx.x];
        __syncthreads();
    }}
    
    if(row < N && col < N) C[row*N + col] = value;
}}
'''

matmul_kernel = cp.RawKernel(kernel_code, 'matmul_tiled')

C_cuda = cp.zeros((N, N), dtype=cp.float32)

threads_per_block = (TILE, TILE)
blocks_per_grid = ((N + TILE - 1)//TILE, (N + TILE - 1)//TILE)

start = time.time()
matmul_kernel(blocks_per_grid, threads_per_block, (A, B, C_cuda, N))
cp.cuda.Stream.null.synchronize()
end = time.time()
time_cuda = end - start
print(f"[CUDA tiled kernel] Time: {time_cuda:.4f} seconds")

# Difference check
diff = cp.max(cp.abs(C_normal - C_cuda))
print(f"Difference between normal and CUDA tiled: {diff:.6f}")

# Show part of result (top-left 5x5 block only)
print("\nSample output (top-left 5x5 block):")
print(C_cuda[:5, :5].get())


[Normal CuPy dot] Time: 1.1330 seconds
[CUDA tiled kernel] Time: 0.1929 seconds
Difference between normal and CUDA tiled: 0.005127

Sample output (top-left 5x5 block):
[[1034.8945  1030.321   1047.7356  1021.0392  1032.5448 ]
 [1030.0343  1030.151   1041.3389  1019.707   1029.7284 ]
 [1024.3102  1027.2953  1041.6548  1028.2279  1029.5394 ]
 [1019.1637  1020.75757 1021.829   1011.0303  1017.7923 ]
 [1018.0107  1008.7152  1039.5358  1015.09216 1020.16864]]
